In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
data_dir = "../data/MDataFiles_Stage1/"

# EXPLORE DATA

**There are a ton of data files when you download from kaggle.  Let's take a look at some of them to see what features we want to use/create for our modelling.**

## Teams

**TeamID is a 4 digit id number from 1000-1999 for the men's teams and a 4 digit id number from 3000-3999 for the women's teams.  These 4 digit id numbers uniquely identify each of NCAA® men's and women's basketball teams. A school's TeamID does not change from one year to the next, so for instance the Duke men's TeamID is 1181 for all seasons.**

In [3]:
team_mens = pd.read_csv("../data/MDataFiles_Stage1/MTeams.csv")
team_mens.shape

(367, 4)

In [4]:
team_mens.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2020
1,1102,Air Force,1985,2020
2,1103,Akron,1985,2020
3,1104,Alabama,1985,2020
4,1105,Alabama A&M,2000,2020


In [5]:
team_mens.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TeamID,367.0,1284.000000,106.088014,1101.0,1192.5,1284.0,1375.5,1467.0
FirstD1Season,367.0,1988.446866,7.632744,1985.0,1985.0,1985.0,1985.0,2020.0
LastD1Season,367.0,2019.117166,4.900642,1985.0,2020.0,2020.0,2020.0,2020.0


- TeamIDs range from 1101 to 1467
- We have data from 1985 to 2020

## Seasons

**This file identifies the different seasons included in the historical data, along with certain season-level properties:**

- Season - indicates the year in which the tournament was played

- DayZero - tells you the date corresponding to daynum=0 during that season. All game dates have been aligned upon a common scale so that the championship game of the final tournament is on daynum=154. Working backward, the national semifinals are always on daynum=152, the "play-in" games are on days 134/135, Selection Sunday is on day 132, and so on. All game data includes the day number in order to make it easier to perform date calculations. If you really want to know the exact date a game was played on, you can combine the game's "daynum" with the season's "dayzero". For instance, since day zero during the 2011-2012 season was 10/31/2011, if we know that the earliest regular season games that year were played on daynum=7, they were therefore played on 11/07/2011.

- RegionW, RegionX, Region Y, Region Z - by convention, the four regions in the final tournament are always named W, X, Y, and Z. 

In [6]:
season = pd.read_csv(data_dir + "MSeasons.csv")
season.shape

(36, 6)

In [7]:
season.head()

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast


In [8]:
season.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Season,36.0,2002.5,10.535654,1985.0,1993.75,2002.5,2011.25,2020.0


- This .csv file contains data from seasons 1985 to 2020 as well.
- We need to reformat "DayZero".
- DayZero - tells you the date corresponding to daynum=0 during that season. All game dates have been aligned upon a common scale so that Selection Monday is on day 133. All game data includes the day number in order to make it easier to perform date calculations. If you really want to know the exact date a game was played on, you can combine the game's "daynum" with the season's "dayzero". For instance, since day zero during the 2011-2012 season was 10/31/2011, if we know that the earliest regular season games that year were played on daynum=7, they were therefore played on 11/07/2011.
- DayNum - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "DayZero" date in the "Seasons.csv" file. For example, the first game in the file was DayNum=20. Combined with the fact from the "Seasons.csv" file that day zero was 10/29/1984 that year, this means the first game was played 20 days later, or 11/18/1984. There are no teams that ever played more than one game on a given date, so you can use this fact if you need a unique key (combining Season and DayNum and WTeamID). In order to accomplish this uniqueness, we had to adjust one game's date. In March 2008, the SEC postseason tournament had to reschedule one game (Georgia-Kentucky) to a subsequent day, so Georgia had to actually play two games on the same day. In order to enforce this uniqueness, we moved the game date for the Georgia-Kentucky game back to its original scheduled date.

In [9]:
season['DayZero'] = pd.to_datetime(season['DayZero'])
season[['DayZero']].describe().transpose()

,count,unique,top,freq,first,last
DayZero,36,36,2017-10-30,1,1984-10-29,2019-11-04


## Seeds

**This file identifies the seeds for all teams in each NCAA® tournament, for all seasons of historical data. Thus, there are between 64-68 rows for each year, depending on whether there were any play-in games and how many there were. In recent years the structure has settled at 68 total teams, with eight "play-in" games leading to the final field of 64 teams entering Round 1 on Thursday of the first week. We do not know the seeds of the respective tournament teams, or even exactly which 68 teams will be, until Selection Sunday every year.**

In [11]:
seed_mens = pd.read_csv(data_dir + "MNCAATourneySeeds.csv")
seed_mens.shape

(2286, 3)

In [12]:
seed_mens.head()

,Season,Seed,TeamID
0,1985,W01,1207
1,1985,W02,1210
2,1985,W03,1228
3,1985,W04,1260
4,1985,W05,1374


In [13]:
seed_mens.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
Season,2286.0,2002.220035,10.139040,1985.0,1993.0,2002.0,2011.0,2019.0
TeamID,2286.0,1291.681977,103.250141,1101.0,1209.0,1287.0,1387.0,1463.0


## Season Results

In [14]:
season_result = pd.read_csv(data_dir + "MRegularSeasonCompactResults.csv")
season_result.shape

(161552, 8)

In [15]:
season_result.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [16]:
season_result.describe().transpose()[['min', 'max']]

,min,max
Season,1985.0,2019.0
DayNum,0.0,132.0
WTeamID,1101.0,1466.0
WScore,34.0,186.0
LTeamID,1101.0,1466.0
LScore,20.0,150.0
NumOT,0.0,6.0


**Looking at the "TEAMS" section above when we fixed the date in the "DayZero" column it had 2019-11-04 00:00:00 for last which doesn;t seem to have the 2020 data - verify...**

In [17]:
print('Seasons without results:')
season.loc[~season['Season'].isin(season_result['Season'])]

Seasons without results:


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
35,2020,2019-11-04,TBD1,TBD2,TBD3,TBD4


**The data for the 2020 season is NOT in the .csv MRegularSeasonCompactResults.csv.**

**Add game date using DayZero + DayNum to get the actual GameDate.**

In [18]:
# Source: https://stackoverflow.com/questions/48210892/pandas-date-difference-using-column-as-offset

def apply_column_offset(df, date_col, offset_col):
    
    df = df.copy()
    df['NewDate'] = df[date_col].values.astype('datetime64[D]') + \
                    df[offset_col].add(1).values.astype('timedelta64[D]') - \
                    np.array([1], dtype='timedelta64[D]')
    return df['NewDate']

season_result = pd.merge(season_result,
                         season[['Season', 'DayZero']],
                         on='Season',
                         how='inner')
            
season_result['GameDate'] = apply_column_offset(season_result, 'DayZero', 'DayNum')
season_result.head(5)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,DayZero,GameDate
0,1985,20,1228,81,1328,64,N,0,1984-10-29,1984-11-18
1,1985,25,1106,77,1354,70,H,0,1984-10-29,1984-11-23
2,1985,25,1112,63,1223,56,H,0,1984-10-29,1984-11-23
3,1985,25,1165,70,1432,54,H,0,1984-10-29,1984-11-23
4,1985,25,1192,86,1447,74,H,0,1984-10-29,1984-11-23


In [19]:
tourney_result = pd.read_csv(data_dir + 'MNCAATourneyCompactResults.csv')
tourney_result.shape

(2251, 8)

In [20]:
tourney_result.head(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
5,1985,136,1246,66,1449,58,N,0
6,1985,136,1256,78,1338,54,N,0
7,1985,136,1260,59,1233,58,N,0
8,1985,136,1314,76,1292,57,N,0
9,1985,136,1323,79,1333,70,N,0


In [21]:
display(tourney_result.describe().transpose()[['min', 'max']])

,min,max
Season,1985.0,2019.0
DayNum,134.0,154.0
WTeamID,1104.0,1463.0
WScore,43.0,149.0
LTeamID,1101.0,1463.0
LScore,29.0,115.0
NumOT,0.0,3.0


In [22]:
print('seasons without results:')
season.loc[~season['Season'].isin(tourney_result['Season'])]

seasons without results:


,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
35,2020,2019-11-04,TBD1,TBD2,TBD3,TBD4


**We can use "DayZero" and "DayNum" together to create the actual date of the game:**

In [23]:
tourney_result = pd.merge(tourney_result,
                          season[['Season', 'DayZero']],
                          on='Season',
                          how='inner')

tourney_result['GameDate'] = apply_column_offset(tourney_result, 'DayZero', 'DayNum')
tourney_result.head(3)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,DayZero,GameDate
0,1985,136,1116,63,1234,54,N,0,1984-10-29,1985-03-14
1,1985,136,1120,59,1345,58,N,0,1984-10-29,1985-03-14
2,1985,136,1207,68,1250,43,N,0,1984-10-29,1985-03-14


In [24]:
sample_submission = pd.read_csv(data_dir + '../MSampleSubmissionStage1_2020.csv')
sample_submission.shape

(11390, 2)

**The sample submission each team has a 50/50 chance of winning any particular game.**

In [25]:
sample_submission.head()

,ID,Pred
0,2015_1107_1112,0.5
1,2015_1107_1116,0.5
2,2015_1107_1124,0.5
3,2015_1107_1125,0.5
4,2015_1107_1129,0.5


In [26]:
sample_submission['Season'] = sample_submission['ID'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Season'].value_counts().sort_index()

2015    2278
2016    2278
2017    2278
2018    2278
2019    2278
Name: Season, dtype: int64

**We use the tournament results from 2015-2019 as our test data to predict on. There was no tournament in 2020 due to COVID-19.**

In [27]:
test = tourney_result.loc[tourney_result['Season'] >= 2015].copy()
test.shape

(335, 10)

In [28]:
test.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,DayZero,GameDate
1916,2015,134,1214,74,1264,64,N,0,2014-11-03,2015-03-17
1917,2015,134,1279,94,1140,90,N,0,2014-11-03,2015-03-17
1918,2015,135,1173,56,1129,55,N,0,2014-11-03,2015-03-18
1919,2015,135,1352,81,1316,77,N,0,2014-11-03,2015-03-18
1920,2015,136,1112,93,1411,72,N,0,2014-11-03,2015-03-19


**Create the ID column with the proper format to submit to kaggle.**

In [29]:
# Source: https://www.kaggle.com/catadanna/delete-leaked-from-training-ncaam-ncaaw-stage1

def create_ID(row):
    if row['WTeamID'] < row['LTeamID']:
        ID = str(row["Season"]) + "_" + str(row["WTeamID"]) + "_" + str(row["LTeamID"])
    else:
        ID = str(row["Season"])+ "_" + str(row["LTeamID"]) + "_" + str(row["WTeamID"])
    return ID

test['ID'] = test.apply(create_ID, axis = 1)
test.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,DayZero,GameDate,ID
1916,2015,134,1214,74,1264,64,N,0,2014-11-03,2015-03-17,2015_1214_1264
1917,2015,134,1279,94,1140,90,N,0,2014-11-03,2015-03-17,2015_1140_1279
1918,2015,135,1173,56,1129,55,N,0,2014-11-03,2015-03-18,2015_1129_1173
1919,2015,135,1352,81,1316,77,N,0,2014-11-03,2015-03-18,2015_1316_1352
1920,2015,136,1112,93,1411,72,N,0,2014-11-03,2015-03-19,2015_1112_1411


In [30]:
print('number of rows with true labels (had actual games):')
sample_submission['ID'].isin(test['ID']).sum()

number of rows with true labels (had actual games):


335

In [31]:
test = sample_submission.copy()

test['Team1'] = test['ID'].apply(lambda x: x.split('_')[1]).astype(int)
test['Team2'] = test['ID'].apply(lambda x: x.split('_')[2]).astype(int)

print(test.shape)
test.head()

(11390, 5)


,ID,Pred,Season,Team1,Team2
0,2015_1107_1112,0.5,2015,1107,1112
1,2015_1107_1116,0.5,2015,1107,1116
2,2015_1107_1124,0.5,2015,1107,1124
3,2015_1107_1125,0.5,2015,1107,1125
4,2015_1107_1129,0.5,2015,1107,1129


**Need to remove the data from years 2015 - 2020 from the training data so there is no leakage.**

In [32]:
train = tourney_result.loc[tourney_result['Season'] < 2015].copy()

rename_dict = {
    'WTeamID': 'Team1',
    'LTeamID': 'Team2',
    'WScore': 'Score1',
    'LScore': 'Score2',
    'WLoc': 'Loc1'
}
train = train.rename(columns=rename_dict)

# target is 1 when Team1 is the winner
train['target'] = 1

print(train.shape)
train.head()

(1916, 11)


,Season,DayNum,Team1,Score1,Team2,Score2,Loc1,NumOT,DayZero,GameDate,target
0,1985,136,1116,63,1234,54,N,0,1984-10-29,1985-03-14,1
1,1985,136,1120,59,1345,58,N,0,1984-10-29,1985-03-14,1
2,1985,136,1207,68,1250,43,N,0,1984-10-29,1985-03-14,1
3,1985,136,1229,58,1425,55,N,0,1984-10-29,1985-03-14,1
4,1985,136,1242,49,1325,38,N,0,1984-10-29,1985-03-14,1


**Add a flipped train set for perspective of other team.**

In [33]:
train_flipped = train.copy()

rename_dict = {
    'Team1': 'Team2',
    'Team2': 'Team1',
    'Score1': 'Score2',
    'Score2': 'Score1'
}

train_flipped = train_flipped.rename(columns=rename_dict)
train_flipped['target'] = 0

train = pd.concat([train, train_flipped], sort=True)
print(train.shape)
train.head(3)

(3832, 11)


,DayNum,DayZero,GameDate,Loc1,NumOT,Score1,Score2,Season,Team1,Team2,target
0,136,1984-10-29,1985-03-14,N,0,63,54,1985,1116,1234,1
1,136,1984-10-29,1985-03-14,N,0,59,58,1985,1120,1345,1
2,136,1984-10-29,1985-03-14,N,0,68,43,1985,1207,1250,1


In [34]:
train_orig = train.copy()
train = train_orig.copy()
team_nums = ['Team1', 'Team2']

**Add the seeds for "Team1" and "Team2" on the training data:**

In [35]:
for i, team_num in enumerate(team_nums):

    train = pd.merge(train, 
                     seed_mens, 
                     left_on=['Season', team_num ], 
                     right_on=['Season', 'TeamID'], 
                     how='left')

    train['var_seed{}'.format(i + 1)] = train['Seed'].apply(lambda x: x[1:3]).astype(int)
    train = train.drop(['Seed', 'TeamID'], axis=1)

print(train.shape)
train.head()

(3832, 13)


,DayNum,DayZero,GameDate,Loc1,NumOT,Score1,Score2,Season,Team1,Team2,target,var_seed1,var_seed2
0,136,1984-10-29,1985-03-14,N,0,63,54,1985,1116,1234,1,9,8
1,136,1984-10-29,1985-03-14,N,0,59,58,1985,1120,1345,1,11,6
2,136,1984-10-29,1985-03-14,N,0,68,43,1985,1207,1250,1,1,16
3,136,1984-10-29,1985-03-14,N,0,58,55,1985,1229,1425,1,9,8
4,136,1984-10-29,1985-03-14,N,0,49,38,1985,1242,1325,1,3,14


**Do the same for the test data:**

In [36]:
for i, team_num in enumerate(team_nums):

    test = pd.merge(test, 
                     seed_mens, 
                     left_on=['Season', team_num ], 
                     right_on=['Season', 'TeamID'], 
                     how='left')

    test['var_seed{}'.format(i + 1)] = test['Seed'].apply(lambda x: x[1:3]).astype(int)
    test = test.drop(['Seed', 'TeamID'], axis=1)

print(test.shape)
test.head()

(11390, 7)


,ID,Pred,Season,Team1,Team2,var_seed1,var_seed2
0,2015_1107_1112,0.5,2015,1107,1112,14,2
1,2015_1107_1116,0.5,2015,1107,1116,14,5
2,2015_1107_1124,0.5,2015,1107,1124,14,3
3,2015_1107_1125,0.5,2015,1107,1125,14,15
4,2015_1107_1129,0.5,2015,1107,1129,14,11


**Create seed difference column on the training data:**

In [37]:
train['var_seed_diff'] = train['var_seed1'] - train['var_seed2']
train.head()

,DayNum,DayZero,GameDate,Loc1,NumOT,Score1,Score2,Season,Team1,Team2,target,var_seed1,var_seed2,var_seed_diff
0,136,1984-10-29,1985-03-14,N,0,63,54,1985,1116,1234,1,9,8,1
1,136,1984-10-29,1985-03-14,N,0,59,58,1985,1120,1345,1,11,6,5
2,136,1984-10-29,1985-03-14,N,0,68,43,1985,1207,1250,1,1,16,-15
3,136,1984-10-29,1985-03-14,N,0,58,55,1985,1229,1425,1,9,8,1
4,136,1984-10-29,1985-03-14,N,0,49,38,1985,1242,1325,1,3,14,-11


**Do the same for the test data:**

In [38]:
test['var_seed_diff'] = test['var_seed1'] - test['var_seed2']
test.head()

,ID,Pred,Season,Team1,Team2,var_seed1,var_seed2,var_seed_diff
0,2015_1107_1112,0.5,2015,1107,1112,14,2,12
1,2015_1107_1116,0.5,2015,1107,1116,14,5,9
2,2015_1107_1124,0.5,2015,1107,1124,14,3,11
3,2015_1107_1125,0.5,2015,1107,1125,14,15,-1
4,2015_1107_1129,0.5,2015,1107,1129,14,11,3


# Modeling

**I am going to create a very simple Logistic Regression model for my first submission.  For this model I am only going to use the seed of the teams and the seed difference between the teams.**

In [39]:
features = [x for x in train.columns if 'var_' in x]
features

['var_seed1', 'var_seed2', 'var_seed_diff']

In [40]:

clf = LogisticRegression(random_state = 69)

# define K-folds
skf = StratifiedKFold(n_splits = 13, 
                      shuffle = True, 
                      random_state = 17)

scores = cross_val_score(clf,
                         train[features],
                         train['target'],
                         scoring='neg_log_loss',
                         cv=skf,
                         n_jobs=6)

print('\n', scores, '\n')

print("log_loss:", round(-scores.mean(), 16))


 [-0.5462549  -0.58429887 -0.57882661 -0.54549195 -0.56151115 -0.53537947
 -0.54037861 -0.56403868 -0.56854713 -0.55607515 -0.56840872 -0.49729305
 -0.54860618] 

log_loss: 0.5534700355931296


In [41]:
test.head()

,ID,Pred,Season,Team1,Team2,var_seed1,var_seed2,var_seed_diff
0,2015_1107_1112,0.5,2015,1107,1112,14,2,12
1,2015_1107_1116,0.5,2015,1107,1116,14,5,9
2,2015_1107_1124,0.5,2015,1107,1124,14,3,11
3,2015_1107_1125,0.5,2015,1107,1125,14,15,-1
4,2015_1107_1129,0.5,2015,1107,1129,14,11,3


**Fit on all the training data...**

In [42]:
clf.fit(train[features], train['target'])

LogisticRegression(random_state=69)

# Create Submission File

**A submission file lists every possible matchup between tournament teams for one or more years. During Stage 1, you are asked to make predictions for all possible matchups from the past four NCAA® tournaments (seasons 2015, 2016, 2017, 2018 and 2019). In Stage 2, you will be asked to make predictions for all possible matchups from the current NCAA® tournament - season 2020.**

**When there are 68 teams in the tournament, there are 68 * 67/2 =2,278 predictions to make for that year, so a Stage 1 submission file will have 2,278 x 5 = 11,390 data rows.**

**ID - this is a 14-character string of the format SSSS_XXXX_YYYY, where SSSS is the four digit season number, XXXX is the four-digit TeamID of the lower-ID team, and YYYY is the four-digit TeamID of the higher-ID team.
Pred - this contains the predicted winning percentage for the first team identified in the ID field**

**For example, if you wanted to make a prediction for Duke (TeamID=1181) against Arizona (TeamID = 1112) in the 2012 tournament, with Duke given a 53% chance to win and Arizona given a 47% chance to win. In this case, Arizona has the lower numerical ID so they would be listed first, and the winning percentage would be expressed from Arizona's perspective (47%):**

**2012_1112_1181,0.47**


In [43]:
submission = pd.DataFrame()
submission['ID'] = test['ID']
submission['pred'] = clf.predict_proba(test[features])[:, 1:]

submission.to_csv('../submissions/submission2.csv', index=False)

submission.shape

(11390, 2)

In [44]:
submission.head()

,ID,pred
0,2015_1107_1112,0.118399
1,2015_1107_1116,0.181568
2,2015_1107_1124,0.137007
3,2015_1107_1125,0.541729
4,2015_1107_1129,0.377089
